In [205]:
import pandas as pd
import numpy as np
import pyspark as spark
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

1) tenemos informacion sobre recetas en 3 RDD de spark

(id_receta, nombre, tiempo_preparacion, dificultad)

In [207]:
Recetas = sc.parallelize([
    (1,"ensalada de pollo", 5, 'facil'),
    (2,"ensalada de atun", 5, 'facil'),
    (3,"ensalada de cordero", 5,'facil'),
    (4,"wok de pollo", 30, 'medio'),
    (5,"wok de cordero", 30, 'medio'),
    (6,"risoto de pollo", 40, 'dificil'),
    (7,"risoto de cordero", 45, 'dificil'),
    (8,"hamburguesa", 30, 'facil'),
    (9,"hamburguesa de cordero", 35, 'facil'),
    (10,"pastel de papa", 40, 'medio'),
    (11,"pizza", 40, 'facil'),
    (12,"tomates rellenos", 10, 'facil'),
    (13,"tortilla de papa", 20, 'medio'),
    (14,"pollo al curry", 30, 'medio'),
    (15,"pasta gratinada", 30, 'medio'),
])

(id_ingredeinte, nombre)

In [208]:
Ingredientes = sc.parallelize([
        (1,"lechuga"),
        (2,"tomate"),
        (3,"pollo"),
        (4,"atun"),
        (5,"cordero"),
        (6,"carne"),
        (7,"arroz"),
        (8,"papa"),
        (9,"pan"),
        (10,"queso"),
        (11,"fideos"),
        (12,"morron"),
        (13,"berenjena"),
        (14,"zanahoria"),
        (15,"masa"),
        (16,"huevo"),
        (17,"curry"),
])

(id_receta, id_ingrediente, cantidad)

In [209]:
Ingredientes_por_receta = sc.parallelize([\
    (1, 1, 1),
    (1, 2, 2),
    (1, 3, 1),                                    
    (2, 1, 1),
    (2, 2, 2),
    (2, 4, 1),                                      
    (3, 1, 1),
    (3, 2, 2),
    (3, 5, 1),                                     
    (4, 3, 1),
    (4, 12, 2),
    (4, 13, 2),
    (4, 14, 3),                                      
    (5, 5, 1),
    (5, 12, 2),
    (5, 13, 2),
    (5, 14, 3),                                      
    (6, 3, 1),
    (6, 7, 1),                                      
    (7, 5, 1),
    (7, 7, 1),                                      
    (8, 6, 1),
    (8, 9, 2),                                      
    (9, 5, 1),
    (9, 9, 2),                                      
    (10, 6, 1),
    (10, 8, 4),                                      
    (11, 15, 1),
    (11, 10, 1),                                      
    (12, 2, 3),
    (12, 10, 1),                                      
    (13, 8, 4),
    (13, 16, 3),                                      
    (14, 3, 1),
    (14, 17, 1),                                      
    (15, 11, 1),
    (15, 10, 1),                                      
])

a) Obtener el nombre de todas las recetas que tengan cordero

In [210]:
#filtro ingredientes por cordero y me quedo con id_ingrediente
ingr = Ingredientes.filter(lambda x: x[1] == 'cordero')
ingr = ingr.map(lambda x: (x[0],0)).collect()
ingr

[(5, 0)]

In [211]:
#me quedo con id_ingrediente y id_receta de ingredientes por receta
ingXreceta = Ingredientes_por_receta.map(lambda x: (x[1],x[0])).collect()
ingXreceta

[(1, 1),
 (2, 1),
 (3, 1),
 (1, 2),
 (2, 2),
 (4, 2),
 (1, 3),
 (2, 3),
 (5, 3),
 (3, 4),
 (12, 4),
 (13, 4),
 (14, 4),
 (5, 5),
 (12, 5),
 (13, 5),
 (14, 5),
 (3, 6),
 (7, 6),
 (5, 7),
 (7, 7),
 (6, 8),
 (9, 8),
 (5, 9),
 (9, 9),
 (6, 10),
 (8, 10),
 (15, 11),
 (10, 11),
 (2, 12),
 (10, 12),
 (8, 13),
 (16, 13),
 (3, 14),
 (17, 14),
 (11, 15),
 (10, 15)]

In [212]:
#hago un join por id_ingrediente
ingXreceta = sc.parallelize(ingXreceta)
ingr = sc.parallelize(ingr)
ing = ingXreceta.join(ingr).collect()
ing

[(5, (3, 0)), (5, (5, 0)), (5, (7, 0)), (5, (9, 0))]

In [213]:
#hago un map para eliminar el 0 de mas
ing = sc.parallelize(ing)
ing = ing.map(lambda x: (x[0],x[1][0])).collect()
ing

[(5, 3), (5, 5), (5, 7), (5, 9)]

In [214]:
#me quedo solo con id_receta
ing = sc.parallelize(ing)
ing = ing.map(lambda x: (x[1],0)).collect()
ing

[(3, 0), (5, 0), (7, 0), (9, 0)]

In [215]:
#me quedo con id_receta y nombre
rec = Recetas.map(lambda x: (x[0],x[1])).collect()
rec

[(1, 'ensalada de pollo'),
 (2, 'ensalada de atun'),
 (3, 'ensalada de cordero'),
 (4, 'wok de pollo'),
 (5, 'wok de cordero'),
 (6, 'risoto de pollo'),
 (7, 'risoto de cordero'),
 (8, 'hamburguesa'),
 (9, 'hamburguesa de cordero'),
 (10, 'pastel de papa'),
 (11, 'pizza'),
 (12, 'tomates rellenos'),
 (13, 'tortilla de papa'),
 (14, 'pollo al curry'),
 (15, 'pasta gratinada')]

In [216]:
#hago join por id_receta
ing = sc.parallelize(ing)
rec = sc.parallelize(rec)
cordero = rec.join(ing).collect()
cordero

[(9, ('hamburguesa de cordero', 0)),
 (3, ('ensalada de cordero', 0)),
 (5, ('wok de cordero', 0)),
 (7, ('risoto de cordero', 0))]

In [217]:
#saco el 0 de mas
cordero1 = sc.parallelize(cordero)
cordero1 = cordero1.map(lambda x: x[1][0]).collect()
cordero1

['hamburguesa de cordero',
 'ensalada de cordero',
 'wok de cordero',
 'risoto de cordero']

b) calcular la cantidad total de cada ingrediente si queremos hacer todas las recetas con cordero que sean faciles.

In [218]:
#saco el 0 de mas de cordero
cordero = sc.parallelize(cordero)
cordero = cordero.map(lambda x: (x[0],x[1][0])).collect()
cordero

[(9, 'hamburguesa de cordero'),
 (3, 'ensalada de cordero'),
 (5, 'wok de cordero'),
 (7, 'risoto de cordero')]

In [219]:
#me quedo solo con recetas faciles
rece = Recetas.filter(lambda x: x[3] == 'facil').collect()
rece

[(1, 'ensalada de pollo', 5, 'facil'),
 (2, 'ensalada de atun', 5, 'facil'),
 (3, 'ensalada de cordero', 5, 'facil'),
 (8, 'hamburguesa', 30, 'facil'),
 (9, 'hamburguesa de cordero', 35, 'facil'),
 (11, 'pizza', 40, 'facil'),
 (12, 'tomates rellenos', 10, 'facil')]

In [220]:
#map por id_receta
rece = sc.parallelize(rece)
rece = rece.map(lambda x: (x[0],0)).collect()
rece

[(1, 0), (2, 0), (3, 0), (8, 0), (9, 0), (11, 0), (12, 0)]

In [221]:
#hago join por id_receta
cordero = sc.parallelize(cordero)
rece = sc.parallelize(rece)
cordero = cordero.join(rece).collect()
cordero

[(9, ('hamburguesa de cordero', 0)), (3, ('ensalada de cordero', 0))]

In [222]:
#saco el 0 de mas
cordero = sc.parallelize(cordero)
cordero = cordero.map(lambda x: (x[0],x[1][0])).collect()
cordero

[(9, 'hamburguesa de cordero'), (3, 'ensalada de cordero')]

In [223]:
#mapeo ingredientes por receta
ingred = Ingredientes_por_receta.map(lambda x: (x[0],(x[1],x[2]))).collect()
ingred

[(1, (1, 1)),
 (1, (2, 2)),
 (1, (3, 1)),
 (2, (1, 1)),
 (2, (2, 2)),
 (2, (4, 1)),
 (3, (1, 1)),
 (3, (2, 2)),
 (3, (5, 1)),
 (4, (3, 1)),
 (4, (12, 2)),
 (4, (13, 2)),
 (4, (14, 3)),
 (5, (5, 1)),
 (5, (12, 2)),
 (5, (13, 2)),
 (5, (14, 3)),
 (6, (3, 1)),
 (6, (7, 1)),
 (7, (5, 1)),
 (7, (7, 1)),
 (8, (6, 1)),
 (8, (9, 2)),
 (9, (5, 1)),
 (9, (9, 2)),
 (10, (6, 1)),
 (10, (8, 4)),
 (11, (15, 1)),
 (11, (10, 1)),
 (12, (2, 3)),
 (12, (10, 1)),
 (13, (8, 4)),
 (13, (16, 3)),
 (14, (3, 1)),
 (14, (17, 1)),
 (15, (11, 1)),
 (15, (10, 1))]

In [224]:
#join con recetas
cordero = sc.parallelize(cordero)
ingred = sc.parallelize(ingred)
cordero = ingred.join(cordero).collect()
cordero


[(9, ((5, 1), 'hamburguesa de cordero')),
 (9, ((9, 2), 'hamburguesa de cordero')),
 (3, ((1, 1), 'ensalada de cordero')),
 (3, ((2, 2), 'ensalada de cordero')),
 (3, ((5, 1), 'ensalada de cordero'))]

In [225]:
#me quedo con id_ingrediente cantidad y receta
cordero = sc.parallelize(cordero)
cordero = cordero.map(lambda x: (x[1][0][0], (x[1][0][1],x[1][1]))).collect()
cordero

[(5, (1, 'hamburguesa de cordero')),
 (9, (2, 'hamburguesa de cordero')),
 (1, (1, 'ensalada de cordero')),
 (2, (2, 'ensalada de cordero')),
 (5, (1, 'ensalada de cordero'))]

In [226]:
#ordeno menor a mayor
cordero = sc.parallelize(cordero)
cordero = cordero.sortByKey(ascending=True).collect()
cordero

[(1, (1, 'ensalada de cordero')),
 (2, (2, 'ensalada de cordero')),
 (5, (1, 'hamburguesa de cordero')),
 (5, (1, 'ensalada de cordero')),
 (9, (2, 'hamburguesa de cordero'))]

In [228]:
#hago un map de ingredientes
Ingredientes = Ingredientes.map(lambda x: (x[0],x[1])).collect()
Ingredientes


[(1, 'lechuga'),
 (2, 'tomate'),
 (3, 'pollo'),
 (4, 'atun'),
 (5, 'cordero'),
 (6, 'carne'),
 (7, 'arroz'),
 (8, 'papa'),
 (9, 'pan'),
 (10, 'queso'),
 (11, 'fideos'),
 (12, 'morron'),
 (13, 'berenjena'),
 (14, 'zanahoria'),
 (15, 'masa'),
 (16, 'huevo'),
 (17, 'curry')]

In [229]:
#hago un join con ingredientes
Ingredientes = sc.parallelize(Ingredientes)
cordero = sc.parallelize(cordero)
cordero = cordero.join(Ingredientes).collect()
cordero

[(1, ((1, 'ensalada de cordero'), 'lechuga')),
 (9, ((2, 'hamburguesa de cordero'), 'pan')),
 (2, ((2, 'ensalada de cordero'), 'tomate')),
 (5, ((1, 'hamburguesa de cordero'), 'cordero')),
 (5, ((1, 'ensalada de cordero'), 'cordero'))]

In [231]:
#me quedo con nombre receta, nombre ingrediente y cantidad
cordero = sc.parallelize(cordero)
cordero = cordero.map(lambda x: (x[1][0][1], x[1][1], x[1][0][0])).collect()
cordero

[('ensalada de cordero', 'lechuga', 1),
 ('hamburguesa de cordero', 'pan', 2),
 ('ensalada de cordero', 'tomate', 2),
 ('hamburguesa de cordero', 'cordero', 1),
 ('ensalada de cordero', 'cordero', 1)]

In [233]:
#ordeno
cordero = sc.parallelize(cordero)
cordero = cordero.sortBy(lambda x: x[0], True).collect()
cordero

[('ensalada de cordero', 'lechuga', 1),
 ('ensalada de cordero', 'tomate', 2),
 ('ensalada de cordero', 'cordero', 1),
 ('hamburguesa de cordero', 'pan', 2),
 ('hamburguesa de cordero', 'cordero', 1)]